In [1]:
%matplotlib nbagg

import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from IPython.display import display, Math, Latex

from sklearn.datasets import load_wine
vins = load_wine()
Data = pd.DataFrame(vins['data'],columns=vins['feature_names'])
Data['class'] = vins['target']

#Data = pd.read_csv('vins.csv',sep=';')


Le jeu de donnée est le résulat d'une analyse chimique de vins produit par 3 cultivateurs. Chaque ligne représente un vin testé où la variable **class** donne numéro du cultivateur qui la produit et les autres variables donnent le résultat de l'analyse chimique de ce vin.

Si vous avez importé les données avec le module sklearn, vous pouvez avoir une description (en anglais) du jeu de donnée en utilisant le code suivant:

In [2]:
print(vins['DESCR'])

Wine Data Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- 1) Alcohol
 		- 2) Malic acid
 		- 3) Ash
		- 4) Alcalinity of ash  
 		- 5) Magnesium
		- 6) Total phenols
 		- 7) Flavanoids
 		- 8) Nonflavanoid phenols
 		- 9) Proanthocyanins
		- 10)Color intensity
 		- 11)Hue
 		- 12)OD280/OD315 of diluted wines
 		- 13)Proline
        	- class:
                - class_0
                - class_1
                - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash: 

(1). Importez les données sous forme d'un dataframe. Familiarisez-vous avec le tableau des données : Vérifiez la taille du tableau, regardez le type des différentes variables etc.

In [ ]:
Data.shape

In [ ]:
Data.dtypes

In [ ]:
Data.index

In [ ]:
Data.info()

In [ ]:
Data.describe()

 ##### Distribution marginale
 ---

(2). Calculez les statistiques d'ordre de la variable **flavanoids**. 

In [ ]:
Data.sort_values(by='flavanoids')['flavanoids']
flavanoids_serie = Data['flavanoids']
pd.DataFrame(np.percentile(flavanoids_serie, q=range(1,100+1)), index=range(1,100+1), columns=['percentile'])
# flavanoids_serie.count()==flavanoids_serie.size


(3). En déduire les quartiles de cette variable. Quelle statistique de dispersion pouvez-vous calculer à partir de vos résultats précédents ? Calculez cette statistique.

Voici les quartiles:

In [ ]:
q25 = np.percentile(flavanoids_serie,25)
Math('q_{{25}}={}'.format(q25))

In [ ]:
q75 = np.percentile(flavanoids_serie,75)
Math('q_{{75}}={}'.format(q75))

Des quartiles, on peut déduire l'écart interquartiles:

In [ ]:
eiq2575=q75-q25
Math('\Delta=q_{{75}}-q_{{25}}={}'.format(eiq2575))

(4). Quelle représentation graphique vous permettrait de répondre à la question 3 ? Affichez ce graphique.

La représentation utile ici est le **boxplot**.

In [ ]:
pd.DataFrame(flavanoids_serie).boxplot()

(5). Calculez la distribution (les valeurs et fréquences) de la variable **flavanoids**. Définissez une fonction qui prends en paramètres un entier r, des valeurs et des fréquences et qui renvoie le moment centré d'ordre r. Utilisez cette fonction pour calculer la variance de la variable **flavanoids** à partir de sa distribution.

Distribution des valeurs:

In [ ]:
flavanoids_distrib = flavanoids_serie.value_counts()
flavanoids_distrib.name = 'distribution'
pd.DataFrame(flavanoids_distrib)

Distribution des fréquences:

In [ ]:
flavanoids_freq = flavanoids_serie.value_counts()/flavanoids_serie.size
flavanoids_freq.name = 'fréquence'
pd.DataFrame(flavanoids_freq)

Fonction qui calcule le moment centré d'ordre r:

In [ ]:
def moment_centre(freq, r=1):
    freq_dropna = freq.dropna()
    freq_index = freq_dropna.index
    freq_values = freq_dropna.values
    freq_list = zip(freq_index, freq_values)
    moment_1 = reduce(lambda x, y: x + y, map(lambda (x, y): x * y, freq_list))
    if r == 1:
        moment_r = moment_1
    else:
        moment_r = reduce(lambda x, y: x + y, map(lambda (x, y): ((x-moment_1)**r) * y, freq_list))
    return round(moment_r,6)


Variance:

In [ ]:
Math('\mbox{{Variance: }} {}'.format(moment_centre(flavanoids_freq,2)))

 ##### Distribution conditionnelle
 ---

(6). Calculez la moyenne conditionnelle et la variance conditionnelle de la variable **alcohol** sachant que la variable **class** vaut 0, 1 et 2. Interpretez les résultats.

Moyenne conditionelle

In [ ]:
class_alcohol = Data[['class','alcohol']]
alcohol_freq_by_class = freq_var1_by_var2(class_alcohol, var1='alcohol', by_var2='class')

alcohol_mean_by_class = [(x,moment_centre(alcohol_freq_by_class[x]))    for x in class_alcohol['class'].unique()]
alcohol_var_by_class  = [(x, moment_centre(alcohol_freq_by_class[x],2)) for x in class_alcohol['class'].unique()]
print('Alcohol by class, moyenne:')
pd.DataFrame(alcohol_mean_by_class, index=['']*len(alcohol_mean_by_class), columns=['class','alcohol'])


Variance conditionelle

In [ ]:
print('Alcohol by class, variance:')
pd.DataFrame(alcohol_var_by_class, index=['']*len(alcohol_mean_by_class), columns=['class','alcohol'])


(7). Affichez sur une même figure l'histogramme de la distribution de la variable **alcohol** sachant que la variable **class** vaut 0, 1 et 2 (conseil: utilisez le paramètre **alpha** de la fonction hist pour mieux voir les superpositions). En quoi cette représentation graphique confirme vos précédents résultats ?

(8). Calculez les fréquences de la variable **alcohol** pour les intervalles (10.,12.5], (12.5,13.5] et (13.5,15.] sachant que la variable **class** vaut 0, 1 et 2. (La notation (a,b] signifie que l'intervalle est ouvert à gauche et fermé à droite)

In [ ]:
# fonction qui calcule un tableau de fréquences conditionnelles à partir d'une DataFrame
def freq_var1_by_var2(dframe, var1, by_var2):
    by_var_unique = dframe[by_var2].unique()
    distrib_by_var = dict(
        [(x, dframe.loc[dframe[by_var2] == x, [by_var2, var1]][var1].value_counts()) for x in by_var_unique])
    size_by_var = dict([(x, dframe.loc[dframe[by_var2] == x, [by_var2, var1]][var1].size) for x in by_var_unique])
    freq_by_var = dict(
        [(x, distrib_by_var[x] / size_by_var[x]) for x in by_var_unique])
    return freq_by_var

alcohol_subserie = {}
alcohol_subserie['(10.,12.5]']  = Data.loc[(Data['alcohol']>10.)&(Data['alcohol']<=12.5), ['class','alcohol']]
alcohol_subserie['(12.5,13.5]'] = Data.loc[(Data['alcohol']>12.5)&(Data['alcohol']<=13.5), ['class','alcohol']]
alcohol_subserie['(13.5,15.]']  = Data.loc[(Data['alcohol']>13.5)&(Data['alcohol']<=15), ['class','alcohol']]

alcohol_subserie_freq = dict([ (x, freq_var1_by_var2(alcohol_subserie[x], 'alcohol', 'class')) for x in alcohol_subserie.keys() ])
# pd.DataFrame(alcohol_subserie_freq['(10.,12.5]'])
alcohol_subserie_freq

 ##### Distribution jointe et statistiques de dépendance
 ---

(9). Affichez le nuage de points des variables **flavanoids** et **total_phenols** et le nuage de points des variables **flavanoids** et **nonflavanoid_phenols** sur des figures différentes (mais dans une même fenêtre). Interpretez ces deux graphiques.

In [ ]:
plt.figure(2)
plt.subplot(121)
plt.subplot(122)
plt.close(1)

(10). Quelle statistique utiliseriez-vous pour justifier votre réponse à la question précédente ? Justifiez votre choix et calculez la valeur de cette statistique pour les deux cas.

(11). À partir des résultats de la question 8, comment feriez-vous pour calculer la distribution jointe des variables **alcohol** et **class** ? (où les valeurs sont 0, 1 et 2 pour la variable class et les intervalles (10.,12.5], (12.5,13.5] et (13.5,15.] pour la variable alcohol) Calculez cette distribution jointe. Faites le produit entre la distribution marginale de la variable **alcohol** et celle de la variable **class**. Qu'est-ce que vous en concluez ?

(12). Ecrivez une fonction **scatter_matrix** qui trace les nuages de points de toutes les variables du tableau de données passé à la fonction en argument, où les graphiques sont organisés en forme de matrice (c'est à dire que le nuage de points des variables i et j se trouve sur la ième ligne et la jème ligne dans la fenêtre). Appliquez cette fonction à votre tableau de données et comparez votre résultat à la matrice de corrélations.

(13). Imaginez que l'on veuille maintenant prédire la variable **class** en fonction des autres variables. Dans ce cas, quelles variables seraient utiles pour faire cette prédiction ? Justifiez votre choix.